In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator().build(number, verbose=False)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count()*5, number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(1000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9968384772449997, 0.9964417454935415)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9995441711081796, 0.9995387515168884)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe(max_rows=10)

In [9]:
y

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
0,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,NaN,Name,NaN,NaN,Error,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
1,ItalianFiscalCode,ItalianVAT,CadastreCode,Error,Plate,Address,NaN,ProvinceCode,NaN,Municipality,...,CountryCode,Name,NaN,NaN,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
2,NaN,NaN,CadastreCode,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,CountryCode,Name,Surname,NaN,EMail,Error,Date,BiologicalSex,Boolean,NumericId
3,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,NaN,Address,NaN,ProvinceCode,Region,NaN,...,NaN,Name,Surname,String,EMail,PhoneNumber,Date,NaN,Boolean,NumericId
4,NaN,ItalianVAT,CadastreCode,Document,Error,NaN,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,NaN,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
5,ItalianFiscalCode,ItalianVAT,NaN,NaN,Plate,Address,NaN,ProvinceCode,Region,Municipality,...,Error,Name,Surname,String,EMail,NaN,Date,BiologicalSex,NaN,NumericId
6,NaN,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,NaN,BiologicalSex,Boolean,NumericId


In [10]:
model.predict_dataframe(X)

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
0,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,NaN,Name,NaN,NaN,Error,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
1,ItalianFiscalCode,ItalianVAT,CadastreCode,Error,Plate,Address,NaN,ProvinceCode,NaN,Municipality,...,CountryCode,Name,NaN,NaN,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
2,NaN,NaN,CadastreCode,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,CountryCode,Name,Surname,NaN,EMail,Error,Date,BiologicalSex,Boolean,NumericId
3,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,NaN,Address,NaN,ProvinceCode,Region,NaN,...,NaN,Name,Surname,String,EMail,PhoneNumber,Date,NaN,Boolean,NumericId
4,NaN,ItalianVAT,CadastreCode,Document,Error,NaN,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,NaN,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
5,ItalianFiscalCode,ItalianVAT,NaN,NaN,Plate,Address,NaN,ProvinceCode,Region,Municipality,...,Error,Name,Surname,String,EMail,NaN,Date,BiologicalSex,NaN,NumericId
6,NaN,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,NaN,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,NaN,BiologicalSex,Boolean,NumericId


In [11]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('Name', 'String'): 1,
         ('BiologicalSex', 'Error'): 3,
         ('Error', 'Document'): 154,
         ('Error', 'ProvinceCode'): 21,
         ('NaN', 'CountryCode'): 99,
         ('Error', 'CadastreCode'): 1,
         ('Error', 'ItalianVAT'): 17,
         ('Error', 'Name'): 14,
         ('Error', 'CountryCode'): 26,
         ('Error', 'NumericId'): 59,
         ('Error', 'ItalianFiscalCode'): 15,
         ('CountryCode', 'NaN'): 62,
         ('String', 'Error'): 256,
         ('Error', 'Surname'): 4,
         ('Error', 'String'): 28,
         ('Error', 'Municipality'): 2,
         ('CountryCode', 'ProvinceCode'): 28,
         ('String', 'Name'): 5,
         ('ItalianZIPCode', 'NumericId'): 282,
         ('Error', 'NaN'): 6,
         ('Error', 'ItalianZIPCode'): 10,
         ('Document', 'Error'): 18,
         ('NumericId', 'Error'): 44,
         ('EMail', 'Error'): 5,
         ('Float', 'Error'): 13,
         ('Error', 'EMail'): 2,
         ('Error', 'Boolean'): 8,
    